# Caderno 10 - Compila todas as métricas.

As tabelas são as combinações dos conjuntos de queries (3 conjuntos) e k = [5, 10, 20].

In [1]:
import pandas as pd

# Modelos
MODELOS = ['Luciano/bert-base-portuguese-cased-finetuned-tcu-acordaos',
           'neuralmind/bert-base-portuguese-cased',
           'neuralmind/bert-large-portuguese-cased',
           'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
           'stjiris/bert-large-portuguese-cased-legal-mlm-sts-v1.0',
           'stjiris/bert-large-portuguese-cased-legal-mlm-nli-sts-v1',    
           'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
           'text-embedding-ada-002',
           'text-embedding-3-small',
           'text-embedding-3-large'           
           ]

# Seleciona o tipo de camada oculta
TIPOS_CAMADA_OCULTA = ['mean_hidden_state']
#TIPOS_CAMADA_OCULTA = ['mean_hidden_state',
#                       'cls_hidden_state']

PASTA_DADOS = './dados/'
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/10_compila_todas_metricas/'
ARQUIVO_METRICAS_BM25 = f'{PASTA_DADOS}outputs/4_metricas_bm25_padrao/metricas_bm25.pickle'
ARQUIVO_METRICAS_ATUAIS = f'{PASTA_DADOS}outputs/3_calcula_metricas_atuais/'
ARQUIVO_METRICAS_ATUAIS_SELECT = f'{ARQUIVO_METRICAS_ATUAIS}metricas_atuais_select.pickle'
ARQUIVO_METRICAS_ATUAIS_SELECT_SWAN = f'{ARQUIVO_METRICAS_ATUAIS}metricas_atuais_select_swan.pickle'
ARQUIVO_METRICAS_ATUAIS_SELECT_SWAN_SYNONYM = f'{ARQUIVO_METRICAS_ATUAIS}metricas_atuais_select_swan_synonym.pickle'
ARQUIVO_MAPA_METRICAS = f'{PASTA_RESULTADO_CADERNO}mapa_metricas.pickle'

## 1. Carrega dados de jurisprudência

In [3]:
import pandas as pd
from metricas import metricas

# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1 (os documentos já estão filtrados)
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

docs, queries, qrels = carrega_juris_tcu()

## 2. Extrai as métricas

Carrega a lista dos resultados das queries realizadas no banco vetorial.

In [5]:
import pickle

# Carrega mapa_metricas
try:
    with open(ARQUIVO_MAPA_METRICAS, 'rb') as arquivo:
        mapa_metricas = pickle.load(arquivo)
except FileNotFoundError:
    mapa_metricas = {} 

# Carrega métricas do bm25_padrao
with open(ARQUIVO_METRICAS_BM25, 'rb') as arquivo:
    metricas_bm25_padrao = pickle.load(arquivo)
    
mapa_metricas['bm25_padrao'] = metricas_bm25_padrao

# Carrega métricas do atuais
with open(ARQUIVO_METRICAS_ATUAIS_SELECT, 'rb') as arquivo:
    metricas_atuais_select = pickle.load(arquivo)
    
with open(ARQUIVO_METRICAS_ATUAIS_SELECT_SWAN, 'rb') as arquivo:
    metricas_atuais_select_swan = pickle.load(arquivo)
    
with open(ARQUIVO_METRICAS_ATUAIS_SELECT_SWAN_SYNONYM, 'rb') as arquivo:
    metricas_atuais_select_swan_synonym = pickle.load(arquivo)
    
mapa_metricas['select'] = metricas_atuais_select
mapa_metricas['select_swan'] = metricas_atuais_select_swan
mapa_metricas['select_swan_synonym'] = metricas_atuais_select_swan_synonym


In [7]:
# Função para encontrar a correspondência entre keys
def find_key(doc_key, docs):
    matches = docs[docs['KEY'].str.contains(doc_key, na=False)]
    if not matches.empty:
        return matches.iloc[0]['KEY']
    return doc_key

# Função que constroi dataframe de resultados no formato esperado pela função de métricas
def processa_resultado(I, docs):
    col_resultado_query_key = []
    col_resultado_doc_key = []
    col_resultado_rank = []

    # Preenchendo as listas
    for query_idx in range(len(I)):
        for rank_idx in range(len(I[0])):
            col_resultado_query_key.append(query_idx + 1)
            col_resultado_doc_key.append(I[query_idx, rank_idx])
            col_resultado_rank.append(rank_idx + 1)

    df_resultados = pd.DataFrame({
        "QUERY_KEY": col_resultado_query_key,
        "DOC_KEY": col_resultado_doc_key,
        "RANK": col_resultado_rank,
    })

    df_resultados['DOC_KEY'] = df_resultados['DOC_KEY'].astype(str)
    df_resultados['DOC_KEY'] = df_resultados['DOC_KEY'].apply(lambda x: find_key(x, docs))
    
    return df_resultados

# Função que verifica se uma chave já existe em um mapa
def verifica_chave(mapa_metricas, chave):
    return chave in mapa_metricas

In [9]:
import pickle
from metricas import metricas
from tqdm import tqdm

for modelo in tqdm(MODELOS, desc='Processando MODELOS'):
    
    caminho_modelo = modelo.split("/")[-1]
    pasta_resultado_busca = f'{PASTA_DADOS}outputs/8_armazena_dados_em_banco_vetorial/{caminho_modelo}/'
    
    for tipo_camada_oculta in tqdm(TIPOS_CAMADA_OCULTA, desc=f'Processando {modelo}', leave=False):
        
        tipo = tipo_camada_oculta.split('_')[0]
        
        # Se as métricas do modelo já está em mapa_metricas continue
        if verifica_chave(mapa_metricas,f'{caminho_modelo}_{tipo}'):
            continue        
        
        arquivo_resultado_busca = f'{pasta_resultado_busca}{caminho_modelo}_{tipo_camada_oculta}_resultado_query.pickle'
        
        with open(arquivo_resultado_busca, 'rb') as arquivo:
            I = pickle.load(arquivo)
            
        df_resultados = processa_resultado(I, docs)
        metrica_calculada = metricas(df_resultados, qrels, aproximacao_trec_eval=True)
        mapa_metricas[f'{caminho_modelo}_{tipo}'] = metrica_calculada

Processando MODELOS:   0%|          | 0/10 [00:00<?, ?it/s]
Processando Luciano/bert-base-portuguese-cased-finetuned-tcu-acordaos:   0%|          | 0/1 [00:00<?, ?it/s]
                                                                                                            
Processando neuralmind/bert-base-portuguese-cased:   0%|          | 0/1 [00:00<?, ?it/s]
                                                                                        
Processando neuralmind/bert-large-portuguese-cased:   0%|          | 0/1 [00:00<?, ?it/s]
                                                                                         
Processando rufimelo/Legal-BERTimbau-sts-large-ma-v3:   0%|          | 0/1 [00:00<?, ?it/s]
                                                                                           
Processando stjiris/bert-large-portuguese-cased-legal-mlm-sts-v1.0:   0%|          | 0/1 [00:00<?, ?it/s]
                                                                          

In [11]:
verifica_chave(mapa_metricas,f'{caminho_modelo}_{tipo}')

True

## 4. Exibe as métricas

In [13]:
# Imprime as métricas para o conjunto de queries 1 (0:50), 2 (100:150), ou 3 (100:150) 
# e para um determinado k (foi gerado para k = 5, 10, 20 e 50.

def compara_metricas(con_query, k):
    # Acumula as métricas
    precisao = []
    recall = []
    mrr = []
    ndcg = []

    for nome in mapa_metricas.keys():
        estatisticas = mapa_metricas[nome][50*(con_query-1):50*(con_query)].describe()
        precisao.append(estatisticas.loc['mean', f'P@{k}'])
        recall.append(estatisticas.loc['mean', f'R@{k}'])
        mrr.append(estatisticas.loc['mean', f'MRR@{k}'])
        ndcg.append(estatisticas.loc['mean', f'nDCG@{k}'])

    df = pd.DataFrame({
        "Modelo": mapa_metricas.keys(),
        f"P@{k}": precisao,
        f"R@{k}": recall,
        f"MRR@{k}": mrr,
        f"nDCG@{k}": ndcg
    })
    return df

def compara_metricas_todas_queries(k):
    # Acumula as métricas
    precisao = []
    recall = []
    mrr = []
    ndcg = []

    for nome in mapa_metricas.keys():
        estatisticas = mapa_metricas[nome].describe()
        precisao.append(estatisticas.loc['mean', f'P@{k}'])
        recall.append(estatisticas.loc['mean', f'R@{k}'])
        mrr.append(estatisticas.loc['mean', f'MRR@{k}'])
        ndcg.append(estatisticas.loc['mean', f'nDCG@{k}'])

    df = pd.DataFrame({
        "Modelo": mapa_metricas.keys(),
        f"P@{k}": precisao,
        f"R@{k}": recall,
        f"MRR@{k}": mrr,
        f"nDCG@{k}": ndcg
    })
    return df

pd.set_option('display.precision', 4)

In [27]:
for con_query in [1, 2, 3]:
    for k in [20]:
        print(f'Resultados para conjunto de query {con_query} e k={k}')
        df_metricas = compara_metricas(con_query, k)
        df_metricas.to_csv(f'{PASTA_RESULTADO_CADERNO}metricas-conj-{con_query}-k-{k}.csv', index=False)
        display(df_metricas)

Resultados para conjunto de query 1 e k=20


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,bm25_padrao,0.209,0.3473,0.5446,0.3525
1,select,0.190,0.3158,0.3624,0.2928
2,select_swan,0.208,0.3462,0.3926,0.3244
3,select_swan_synonym,0.218,0.3616,0.4043,0.3415
4,bert-base-portuguese-cased-finetuned-tcu-acord...,0.035,0.0567,0.1824,0.0649
5,bert-base-portuguese-cased-finetuned-tcu-acord...,0.052,0.0811,0.2163,0.0861
6,bert-base-portuguese-cased_mean,0.032,0.0499,0.1849,0.0648
7,bert-base-portuguese-cased_cls,0.037,0.0571,0.1180,0.0536
8,bert-large-portuguese-cased_mean,0.070,0.1169,0.2110,0.1112
9,bert-large-portuguese-cased_cls,0.027,0.0451,0.1598,0.0515


Resultados para conjunto de query 2 e k=20


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,bm25_padrao,0.271,0.4538,0.8665,0.5618
1,select,0.143,0.2384,0.7948,0.4058
2,select_swan,0.156,0.2596,0.8683,0.4442
3,select_swan_synonym,0.157,0.2611,0.8667,0.4457
4,bert-base-portuguese-cased-finetuned-tcu-acord...,0.077,0.1291,0.3644,0.1487
5,bert-base-portuguese-cased-finetuned-tcu-acord...,0.054,0.0874,0.2519,0.1022
6,bert-base-portuguese-cased_mean,0.076,0.1275,0.3280,0.1390
7,bert-base-portuguese-cased_cls,0.038,0.0638,0.1932,0.0669
8,bert-large-portuguese-cased_mean,0.104,0.1744,0.4200,0.2041
9,bert-large-portuguese-cased_cls,0.017,0.0288,0.0648,0.0283


Resultados para conjunto de query 3 e k=20


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,bm25_padrao,0.269,0.4762,0.9175,0.5867
1,select,0.003,0.0053,0.0410,0.0111
2,select_swan,0.009,0.0164,0.1100,0.0364
3,select_swan_synonym,0.009,0.0164,0.1100,0.0364
4,bert-base-portuguese-cased-finetuned-tcu-acord...,0.152,0.2695,0.6134,0.3266
5,bert-base-portuguese-cased-finetuned-tcu-acord...,0.085,0.1514,0.3652,0.1829
6,bert-base-portuguese-cased_mean,0.134,0.2396,0.6066,0.3045
7,bert-base-portuguese-cased_cls,0.044,0.0789,0.1809,0.0888
8,bert-large-portuguese-cased_mean,0.145,0.2562,0.6126,0.3145
9,bert-large-portuguese-cased_cls,0.031,0.0560,0.1450,0.0614


In [29]:
for k in [20]:
    df_metricas = compara_metricas_todas_queries(k)
    df_metricas.to_csv(f'{PASTA_RESULTADO_CADERNO}metricas-k-{k}.csv', index=False)
    display(df_metricas)

,Modelo,P@20,R@20,MRR@20,nDCG@20
0,bm25_padrao,0.2497,0.4258,0.7762,0.5004
1,select,0.1120,0.1865,0.3994,0.2366
2,select_swan,0.1243,0.2074,0.4570,0.2683
3,select_swan_synonym,0.1280,0.2130,0.4603,0.2746
4,bert-base-portuguese-cased-finetuned-tcu-acord...,0.0880,0.1518,0.3867,0.1801
5,bert-base-portuguese-cased-finetuned-tcu-acord...,0.0637,0.1066,0.2778,0.1237
6,bert-base-portuguese-cased_mean,0.0807,0.1390,0.3732,0.1695
7,bert-base-portuguese-cased_cls,0.0397,0.0666,0.1640,0.0698
8,bert-large-portuguese-cased_mean,0.1063,0.1825,0.4146,0.2099
9,bert-large-portuguese-cased_cls,0.0250,0.0433,0.1232,0.0471


In [31]:
# Gravando mapa_metricas
caminho_arquivo = f'{PASTA_RESULTADO_CADERNO}mapa_metricas.pickle'
with open(caminho_arquivo, 'wb') as arquivo_pickle:
    pickle.dump(mapa_metricas, arquivo_pickle)

In [15]:
mapa_metricas

{'bm25_padrao':      QUERY_KEY  P@5  P@10  P@20  P@50     R@5    R@10    R@20    R@50  MRR@5  \
 0            1  0.6   0.3  0.25  0.16  0.2000  0.2000  0.3333  0.5333    1.0   
 1            2  0.4   0.4  0.25  0.12  0.2857  0.5714  0.7143  0.8571    1.0   
 2            3  0.0   0.0  0.00  0.06  0.0000  0.0000  0.0000  0.2000    0.0   
 3            4  0.2   0.3  0.25  0.20  0.0769  0.2308  0.3846  0.7692    0.5   
 4            5  0.0   0.1  0.15  0.08  0.0000  0.0769  0.2308  0.3077    0.0   
 ..         ...  ...   ...   ...   ...     ...     ...     ...     ...    ...   
 145        146  0.4   0.2  0.15  0.08  0.1667  0.1667  0.2500  0.3333    1.0   
 146        147  0.2   0.1  0.15  0.06  0.0714  0.0714  0.2143  0.2143    1.0   
 147        148  0.4   0.3  0.25  0.14  0.1538  0.2308  0.3846  0.5385    1.0   
 148        149  0.2   0.4  0.30  0.18  0.0833  0.3333  0.5000  0.7500    1.0   
 149        150  0.8   0.5  0.35  0.16  0.3636  0.4545  0.6364  0.7273    1.0   
 
      MRR@1